# Multi-Agent Sales Outreach System

This notebook implements a sophisticated multi-agent system for generating and sending sales outreach emails. It utilizes a hierarchical structure where a "Sales Manager" agent orchestrates three specialized drafting agents (Professional, Engaging, Busy) and selects the best output before handing off to an "Email Manager" for formatting and delivery.

In [ ]:
# Import dependencies
from dotenv import load_dotenv
from agents import Agent, Runner, trace, function_tool
from openai.types.responses import ResponseTextDeltaEvent
from typing import Dict
import sendgrid
import os
from sendgrid.helpers.mail import Mail, Email, To, Content
import asyncio

In [ ]:
# Initialize Environment
load_dotenv(override=True)

In [ ]:
# Define Agent Personas

instructions1 = "You are a sales agent working for ComplAI, a company that provides a SaaS tool for ensuring SOC2 compliance. You write professional, serious cold emails."
instructions2 = "You are a humorous, engaging sales agent working for ComplAI. You write witty, engaging cold emails that are likely to get a response."
instructions3 = "You are a busy sales agent working for ComplAI. You write concise, to-the-point cold emails."

sales_agent1 = Agent(name="Professional Sales Agent", instructions=instructions1, model="gpt-4o-mini")
sales_agent2 = Agent(name="Engaging Sales Agent", instructions=instructions2, model="gpt-4o-mini")
sales_agent3 = Agent(name="Busy Sales Agent", instructions=instructions3, model="gpt-4o-mini")

In [ ]:
# Parallel Execution Example
async def generate_parallel_drafts():
    message = "Write a cold sales email"
    with trace("Parallel Draft Generation"):
        results = await asyncio.gather(
            Runner.run(sales_agent1, message),
            Runner.run(sales_agent2, message),
            Runner.run(sales_agent3, message),
        )
    
    outputs = [result.final_output for result in results]
    for i, output in enumerate(outputs):
        print(f"--- Draft {i+1} ---\n{output}\n")

await generate_parallel_drafts()

In [ ]:
# Define Email Sending Tool
@function_tool
def send_email(body: str):
    """ Send out an email with the given body to all sales prospects """
    api_key = os.environ.get('SENDGRID_API_KEY')
    if not api_key:
        return {"status": "simulated", "message": "Email sending simulated (No API Key)"}
        
    sg = sendgrid.SendGridAPIClient(api_key=api_key)
    from_email = Email("ed@edwarddonner.com")  # Replace with verified sender
    to_email = To("ed.donner@gmail.com")  # Replace with recipient
    content = Content("text/plain", body)
    mail = Mail(from_email, to_email, "Sales email", content).get()
    try:
        sg.client.mail.send.post(request_body=mail)
        return {"status": "success"}
    except Exception as e:
        return {"status": "error", "message": str(e)}

In [ ]:
# Convert Agents to Tools
description = "Write a cold sales email"
tool1 = sales_agent1.as_tool(tool_name="sales_agent1", tool_description=description)
tool2 = sales_agent2.as_tool(tool_name="sales_agent2", tool_description=description)
tool3 = sales_agent3.as_tool(tool_name="sales_agent3", tool_description=description)

drafting_tools = [tool1, tool2, tool3]

In [ ]:
# Define Downstream Email Manager Agent
subject_instructions = "Write a compelling subject line for a cold sales email."
html_instructions = "Convert a text email body to a clean, professional HTML layout."

subject_writer = Agent(name="Subject Writer", instructions=subject_instructions, model="gpt-4o-mini")
subject_tool = subject_writer.as_tool(tool_name="subject_writer", tool_description="Write a subject line")

html_converter = Agent(name="HTML Converter", instructions=html_instructions, model="gpt-4o-mini")
html_tool = html_converter.as_tool(tool_name="html_converter", tool_description="Convert text to HTML")

@function_tool
def send_html_email(subject: str, html_body: str) -> Dict[str, str]:
    """ Send out an HTML email """
    # (Implementation similar to send_email, omitted for brevity)
    print(f"[Simulated] Sending Email: {subject}")
    return {"status": "success"}

email_manager_tools = [subject_tool, html_tool, send_html_email]

email_manager = Agent(
    name="Email Manager",
    instructions="Receive an email body, add a subject, convert to HTML, and send it.",
    tools=email_manager_tools,
    model="gpt-4o-mini",
    handoff_description="Format and send the final email"
)

In [ ]:
# Define Orchestrator Agent (Sales Manager)
manager_instructions = """
You are a Sales Manager. Your goal is to find the single best cold sales email.
1. Use all three sales_agent tools to generate drafts.
2. Select the best one based on effectiveness.
3. Hand off the winning draft to the 'Email Manager' agent for delivery.
"""

sales_manager = Agent(
    name="Sales Manager",
    instructions=manager_instructions,
    tools=drafting_tools,
    handoffs=[email_manager],
    model="gpt-4o-mini"
)

In [ ]:
# Execute Full Workflow
async def run_workflow():
    message = "Send out a cold sales email addressed to 'Dear CEO' from Alice"
    with trace("Sales Workflow"):
        result = await Runner.run(sales_manager, message)
        print(result.final_output)

await run_workflow()